<a href="https://colab.research.google.com/github/lee-jaewook/2021_ai_project_house_price_prediction/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인공신경망을 이용한 주택가격지수 예측

In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import plot_model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
import tensorflow as tf
import pydot
import graphviz 
from numpy import unique
from numpy import argmax
import math
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AI 2021/Dataset/house_dataset(micro_macro).csv")

In [5]:
for i in range(df.shape[0]):
    df["Date"][i]=df["Date"][i][0:4]+df["Date"][i][6:-1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
X = np.array(df.iloc[:,1:-1])
y = np.array(df.iloc[:,-1])

In [7]:
X.shape

(168, 12)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

### Train : 0~143( ~ 2018)
### test : 144~ (2019 ~)

In [9]:
X_micro_train=X[:144,:6]
X_micro_test=X[144:,:6]
X_macro_train=X[:144,6:]
X_macro_test=X[144:,6:]

y_train = y[:144]
y_test = y[144:]

# Macro data (CASE 1)
---

In [10]:
look_back = 6
trainX = np.reshape(X_macro_train, (X_macro_train.shape[0], 1, X_macro_train.shape[1]))
testX = np.reshape(X_macro_test, (X_macro_test.shape[0], 1, X_macro_test.shape[1]))

In [11]:
print(trainX.shape)

(144, 1, 6)


In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(1, look_back), return_sequences=True))
model.add(LSTM(32, input_shape=(1, look_back), return_sequences=True))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, y_train, epochs=10000, batch_size=32, verbose=2)

# evaluate on test set
preds = model.predict(testX)

preds1=np.reshape(preds,(preds.shape[0],))

case1 = np.concatenate((y_train,preds1),axis=0)

tf.keras.backend.clear_session()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 7501/10000
5/5 - 0s - loss: 0.0421
Epoch 7502/10000
5/5 - 0s - loss: 0.0402
Epoch 7503/10000
5/5 - 0s - loss: 0.0479
Epoch 7504/10000
5/5 - 0s - loss: 0.0444
Epoch 7505/10000
5/5 - 0s - loss: 0.0311
Epoch 7506/10000
5/5 - 0s - loss: 0.0317
Epoch 7507/10000
5/5 - 0s - loss: 0.0308
Epoch 7508/10000
5/5 - 0s - loss: 0.0267
Epoch 7509/10000
5/5 - 0s - loss: 0.0355
Epoch 7510/10000
5/5 - 0s - loss: 0.0342
Epoch 7511/10000
5/5 - 0s - loss: 0.0210
Epoch 7512/10000
5/5 - 0s - loss: 0.0211
Epoch 7513/10000
5/5 - 0s - loss: 0.0189
Epoch 7514/10000
5/5 - 0s - loss: 0.0199
Epoch 7515/10000
5/5 - 0s - loss: 0.0185
Epoch 7516/10000
5/5 - 0s - loss: 0.0188
Epoch 7517/10000
5/5 - 0s - loss: 0.0145
Epoch 7518/10000
5/5 - 0s - loss: 0.0129
Epoch 7519/10000
5/5 - 0s - loss: 0.0139
Epoch 7520/10000
5/5 - 0s - loss: 0.0149
Epoch 7521/10000
5/5 - 0s - loss: 0.0167
Epoch 7522/10000
5/5 - 0s - loss: 0.0143
Epoch 7523/10000
5/5 - 0s - loss: 0.0159
Epoch 7524/10000
5/5 

# Micro data (CASE 2)
---

In [13]:
look_back = 6
trainX = np.reshape(X_micro_train, (X_micro_train.shape[0], 1, X_micro_train.shape[1]))
testX = np.reshape(X_micro_test, (X_micro_test.shape[0], 1, X_micro_test.shape[1]))

In [14]:
model = Sequential()
model.add(LSTM(128, input_shape=(1, look_back), return_sequences=True))
model.add(LSTM(32, input_shape=(1, look_back), return_sequences=True))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, y_train, epochs=10000, batch_size=32, verbose=2)

# evaluate on test set
preds = model.predict(testX)

preds2=np.reshape(preds,(preds.shape[0],))

case2 = np.concatenate((y_train,preds2),axis=0)

tf.keras.backend.clear_session()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 7501/10000
5/5 - 0s - loss: 6.1564e-04
Epoch 7502/10000
5/5 - 0s - loss: 6.2409e-04
Epoch 7503/10000
5/5 - 0s - loss: 7.9840e-04
Epoch 7504/10000
5/5 - 0s - loss: 6.1852e-04
Epoch 7505/10000
5/5 - 0s - loss: 7.9701e-04
Epoch 7506/10000
5/5 - 0s - loss: 0.0011
Epoch 7507/10000
5/5 - 0s - loss: 8.0614e-04
Epoch 7508/10000
5/5 - 0s - loss: 7.9697e-04
Epoch 7509/10000
5/5 - 0s - loss: 7.0179e-04
Epoch 7510/10000
5/5 - 0s - loss: 8.2207e-04
Epoch 7511/10000
5/5 - 0s - loss: 9.1669e-04
Epoch 7512/10000
5/5 - 0s - loss: 9.9768e-04
Epoch 7513/10000
5/5 - 0s - loss: 7.1868e-04
Epoch 7514/10000
5/5 - 0s - loss: 9.6694e-04
Epoch 7515/10000
5/5 - 0s - loss: 0.0010
Epoch 7516/10000
5/5 - 0s - loss: 0.0014
Epoch 7517/10000
5/5 - 0s - loss: 0.0015
Epoch 7518/10000
5/5 - 0s - loss: 0.0016
Epoch 7519/10000
5/5 - 0s - loss: 0.0027
Epoch 7520/10000
5/5 - 0s - loss: 0.0035
Epoch 7521/10000
5/5 - 0s - loss: 0.0050
Epoch 7522/10000
5/5 - 0s - loss: 0.0039
Epoch 7523

# All data (CASE 3)
---

In [15]:
look_back = 12
X_train=X[:144,]
X_test=X[144:,]
trainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
testX = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [16]:
X_train.shape

(144, 12)

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(1, look_back), return_sequences=True))
model.add(LSTM(32, input_shape=(1, look_back), return_sequences=True))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, y_train, epochs=10000, batch_size=32, verbose=2)

# evaluate on test set
preds = model.predict(testX)

preds3=np.reshape(preds,(preds.shape[0],))

case3 = np.concatenate((y_train,preds3),axis=0)

tf.keras.backend.clear_session()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5/5 - 0s - loss: 0.0044
Epoch 3457/10000
5/5 - 0s - loss: 0.0036
Epoch 3458/10000
5/5 - 0s - loss: 0.0045
Epoch 3459/10000
5/5 - 0s - loss: 0.0041
Epoch 3460/10000
5/5 - 0s - loss: 0.0044
Epoch 3461/10000
5/5 - 0s - loss: 0.0049
Epoch 3462/10000
5/5 - 0s - loss: 0.0049
Epoch 3463/10000
5/5 - 0s - loss: 0.0039
Epoch 3464/10000
5/5 - 0s - loss: 0.0039
Epoch 3465/10000
5/5 - 0s - loss: 0.0036
Epoch 3466/10000
5/5 - 0s - loss: 0.0031
Epoch 3467/10000
5/5 - 0s - loss: 0.0050
Epoch 3468/10000
5/5 - 0s - loss: 0.0033
Epoch 3469/10000
5/5 - 0s - loss: 0.0035
Epoch 3470/10000
5/5 - 0s - loss: 0.0039
Epoch 3471/10000
5/5 - 0s - loss: 0.0048
Epoch 3472/10000
5/5 - 0s - loss: 0.0046
Epoch 3473/10000
5/5 - 0s - loss: 0.0041
Epoch 3474/10000
5/5 - 0s - loss: 0.0041
Epoch 3475/10000
5/5 - 0s - loss: 0.0040
Epoch 3476/10000
5/5 - 0s - loss: 0.0040
Epoch 3477/10000
5/5 - 0s - loss: 0.0044
Epoch 3478/10000
5/5 - 0s - loss: 0.0051
Epoch 3479/10000
5/5 - 0s - loss: 0.00

# matplot graph

In [ ]:
plt.figure(figsize=(12, 6))
x_date = df.iloc[:,0]
plt.plot(x_date,case1, color="salmon")
plt.plot(x_date,case2, color="seagreen")
plt.plot(x_date,case3, color="deeppink")
plt.plot(x_date,y, color="black")
plt.legend(['case 1','case 2','case 3', 'real'])
plt.xticks(ticks=x_date, rotation=45)
plt.locator_params(axis='x', nbins=x_date.shape[0]/12)
plt.show()

In [ ]:
arr=[mean_squared_error(y_test,preds1), mean_squared_error(y_test,preds2), mean_squared_error(y_test,preds3)]
result=pd.DataFrame(arr).transpose()

In [ ]:
result

In [ ]:
import os
if not os.path.exists('/content/drive/MyDrive/Colab Notebooks/AI 2021/result_lstm.csv'):
    result.to_csv('/content/drive/MyDrive/Colab Notebooks/AI 2021/result_lstm.csv', index=False, mode='w', encoding='utf-8-sig')
else:
    result.to_csv('/content/drive/MyDrive/Colab Notebooks/AI 2021/result_lstm.csv', index=False, mode='a', encoding='utf-8-sig', header=False)